### spark session and libraries

In [1]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql import functions
from pyspark.sql.functions import col, udf
from IPython.display import display
from datetime import datetime
from pyspark.sql.types import FloatType, BooleanType, IntegerType
from time import mktime
from datetime import datetime
import pandas as pd

spark_conf = SparkConf() \
    .setAll([
         ['spark.serializer','org.apache.spark.serializer.KryoSerializer'],
         ['spark.rdd.compress','true'],   
    ])

spark = SparkSession \
    .builder \
    .appName("pager") \
    .config(conf=spark_conf) \
    .enableHiveSupport() \
    .getOrCreate()

# ETL

### create dataframe and drop low level features
save the processed file as parquet since it is a columnar format we can perfrom groupby operations faster.

In [2]:
# read 311_service_requests data from hdfs
df = spark.read.csv('hdfs://ip-172-31-51-46.ec2.internal/pager/311_service_requests.csv', header=True,
                   inferSchema=True, ignoreLeadingWhiteSpace=None, ignoreTrailingWhiteSpace=None)

# rename columns and convert to lower case
for col, dtype in df.dtypes:
    new_col = col.replace(" ", "")
    df = df.withColumnRenamed(col, new_col)
    if dtype == 'string':
        df = df.withColumn(new_col, functions.lower(df[new_col]))



df = df.dropna(how='any', subset=['CreatedDate','ClosedDate'])

# UDF fuctions
get_month_func = udf(lambda x: datetime.strptime(x, '%m/%d/%Y %I:%M:%S %p').month, IntegerType())
get_year_func = udf(lambda x: datetime.strptime(x, '%m/%d/%Y %I:%M:%S %p').year, IntegerType())
time_difference_func = udf(lambda x, y: (datetime.strptime(x, '%m/%d/%Y %I:%M:%S %p') - 
                                         datetime.strptime(y, '%m/%d/%Y %I:%M:%S %p')).total_seconds()/3600, 
                           FloatType())
is_school_func = udf(lambda x: True if x == "Unspecified" else False, BooleanType())


# create new columns and select required columns
df = df.withColumn('Month', get_month_func(df['CreatedDate']))
df = df.withColumn('Year', get_year_func(df['CreatedDate']))
df = df.withColumn('TimeTaken', time_difference_func(df['ClosedDate'], df['CreatedDate']))
df = df.withColumn('SchoolZone', is_school_func(df['SchoolName']))

# filters
df.filter(df['TimeTaken']>0)
df = df[['UniqueKey', 'Month', 'Year', 'TimeTaken', 'Agency', 'ComplaintType', 'Descriptor', 'LocationType',
         'Incidentzip', 'AddressType', 'City', 'FacilityType', 'Borough', 'Status', 'SchoolZone', 'CreatedDate',
         'ClosedDate']]

df.write.parquet('hdfs://ip-172-31-51-46.ec2.internal/pager/parquet/311_data', mode='overwrite')

# Exploratory Analysis

 Group based on the each of the selected column for selected column + Time Taken column pair
 
 For each grouping use aggregate for count and mean
 
 Convert the grouped data into pandas dataframes and write into excel sheets
 
 Perfrom exploratory analysis means of grouped data by joining all the pandas dataframes formed
 
 Check for variance in the means
 
  (i) Large variance/stddev implies that the feature is important driver because it means that in each 
      feature the category are diverse and have extreme varying effects

In [8]:
df = spark.read.parquet("hdfs://ip-172-31-51-46.ec2.internal/pager/parquet/311_data")

In [4]:
grouped_counts_dict = {}
pandas_df_dict = {}

select_columns = ['Agency', 'ComplaintType', 'LocationType', 'AddressType', 'City', 'FacilityType', 'Borough', 
                  'Status', 'SchoolZone']

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('./pager_analysis.xlsx', engine='xlsxwriter')

for col in select_columns:
    group = [col] + ['TimeTaken']
    grouped = df.select(group).groupBy(col)
    grouped_counts_dict[col] = {}
    grouped_counts_dict[col]['counts'] = grouped.count()
    grouped_counts_dict[col]['mean_time_taken'] = grouped.mean('TimeTaken')

    pandas_df_count = grouped_counts_dict[col]['counts'].toPandas()
    pandas_df_mean = grouped_counts_dict[col]['mean_time_taken'].toPandas()
    
    pandas_df = pandas_df_count.join(pandas_df_mean.set_index(col), on=col)
    pandas_df_dict[col] = pandas_df[['avg(TimeTaken)']].describe()
    
    pandas_df.to_excel(writer, sheet_name=col)

exploratory_analysis = pd.concat([pandas_df_dict[col] for col in select_columns], axis=1).reset_index()
exploratory_analysis.columns = ['TimeTaken'] + select_columns
exploratory_analysis.to_excel(writer, sheet_name='exploratory_analysis')

display(exploratory_analysis)

,TimeTaken,Agency,ComplaintType,LocationType,AddressType,City,FacilityType,Borough,Status,SchoolZone
0,count,2.900000e+01,281.000000,153.000000,6.000000,2282.000000,6.000000,549.000000,22.000000,1.000000
1,mean,-2.408990e+05,-30254.601041,-65967.413321,-62.621543,87.751361,-119119.589022,295.882093,-180390.380575,-46.657091
2,std,4.341196e+05,151318.642783,239170.313188,364.725634,14132.847890,293709.568884,693.739331,372479.377116,NaN
3,min,-1.028939e+06,-986899.875000,-977421.773707,-754.161199,-645431.919074,-718644.858181,-271.246716,-991193.293689,-46.657091
4,25%,-1.478868e+04,3.187477,4.335747,-90.911113,316.186239,-43.495131,64.766670,-3271.359866,-46.657091
5,50%,8.517787e+01,126.508135,183.532326,38.834693,371.210968,-7.003123,124.000000,32.418155,-46.657091
6,75%,3.077275e+02,471.247654,631.201495,160.169901,387.588823,311.472515,235.449997,180.691970,-46.657091
7,max,1.539200e+03,16222.077081,2198.201251,234.307302,8398.497027,3579.358369,8135.866699,6093.748832,-46.657091


## Do the top features change over time

Group based on Year/Month and the column

Aggregate on the TimeTaken column (mean, counts)

convert the result into pandas dataframe and write to xlsx file

use the excel file to check if the mean is changing with year/month (time)

### Yearly

In [9]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('./pager_yearly_analysis.xlsx', engine='xlsxwriter')

for col in select_columns:
    group = [col] + ['TimeTaken', 'Year']
    grouped = df.select(group).groupBy(['Year'] + [col])
    grouped_counts_dict[col] = {}
    grouped_counts_dict[col]['counts'] = grouped.count()
    grouped_counts_dict[col]['mean_time_taken'] = grouped.mean('TimeTaken')

    pandas_df_count = grouped_counts_dict[col]['counts'].toPandas()
    pandas_df_mean = grouped_counts_dict[col]['mean_time_taken'].toPandas()
    
    pandas_df = pandas_df_count.join(pandas_df_mean.set_index(['Year'] + [col]), 
                                     on=['Year'] + [col]).sort_values(by=col).reset_index(drop=True)
    pandas_df.to_excel(writer, sheet_name=col)

### Monthly

In [10]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('./pager_monthly_analysis.xlsx', engine='xlsxwriter')

for col in select_columns:
    group = [col] + ['TimeTaken', 'Month']
    grouped = df.select(group).groupBy(['Month'] + [col])
    grouped_counts_dict[col] = {}
    grouped_counts_dict[col]['counts'] = grouped.count()
    grouped_counts_dict[col]['mean_time_taken'] = grouped.mean('TimeTaken')

    pandas_df_count = grouped_counts_dict[col]['counts'].toPandas()
    pandas_df_mean = grouped_counts_dict[col]['mean_time_taken'].toPandas()
    
    pandas_df = pandas_df_count.join(pandas_df_mean.set_index(['Month'] + [col]), 
                                     on=['Month'] + [col]).sort_values(by=col).reset_index(drop=True)
    pandas_df.to_excel(writer, sheet_name=col)